In [1]:
import cobra_postgres
import cobra_datamodelfabric
import uuid
import time

In [6]:
pg = cobra_postgres.PgInterface()

In [7]:
pg.create_schema('gdal', True)

In [8]:
import cobra_gdal

In [9]:
gdal = cobra_gdal.cobra_gdal()

In [10]:
gdal.init_database_structure()

In [11]:
files = ['gis_osm_buildings_a_free_1.shp',
'gis_osm_landuse_a_free_1.shp',
'gis_osm_natural_a_free_1.shp',
'gis_osm_natural_free_1.shp',
'gis_osm_places_a_free_1.shp',
'gis_osm_places_free_1.shp',
'gis_osm_pofw_a_free_1.shp',
'gis_osm_pofw_free_1.shp',
'gis_osm_pois_a_free_1.shp',
'gis_osm_pois_free_1.shp',
'gis_osm_railways_free_1.shp',
'gis_osm_roads_free_1.shp',
'gis_osm_traffic_a_free_1.shp',
'gis_osm_traffic_free_1.shp',
'gis_osm_transport_a_free_1.shp',
'gis_osm_transport_free_1.shp',
'gis_osm_water_a_free_1.shp',
'gis_osm_waterways_free_1.shp']

In [12]:
for a_file in files:
    print(a_file)
    gdal.create_shape2pg_job(f'/download/{a_file}')

gis_osm_buildings_a_free_1.shp
gis_osm_landuse_a_free_1.shp
gis_osm_natural_a_free_1.shp
gis_osm_natural_free_1.shp
gis_osm_places_a_free_1.shp
gis_osm_places_free_1.shp
gis_osm_pofw_a_free_1.shp
gis_osm_pofw_free_1.shp
gis_osm_pois_a_free_1.shp
gis_osm_pois_free_1.shp
gis_osm_railways_free_1.shp
gis_osm_roads_free_1.shp
gis_osm_traffic_a_free_1.shp
gis_osm_traffic_free_1.shp
gis_osm_transport_a_free_1.shp
gis_osm_transport_free_1.shp
gis_osm_water_a_free_1.shp
gis_osm_waterways_free_1.shp


In [18]:
gdal.pick_a_job_when_not_busy()

In [19]:
for i in range(20):
    time.sleep(30)
    gdal.pick_a_job_when_not_busy()

KeyboardInterrupt: 